In [11]:
# Importar libreas y datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Enlace de Google Drive con datos
url = 'https://docs.google.com/spreadsheets/d/1OW2aSXdJwDYEp1igJQe67Pcd7Kdyl6XL/edit?usp=sharing&ouid=114907369248611279988&rtpof=true&sd=true'

# Leer el archivo CSV directamente desde Google Drive
df = pd.read_csv(url, delimiter=';')

REGION_DEPTO = {
    'ATLANTICO': 'Caribe',
    'BOLIVAR': 'Caribe',
    'CESAR': 'Caribe',
    'CORDOBA': 'Caribe',
    'SUCRE': 'Caribe',
    'SAN ANDRES': 'Caribe',
    'CAUCA': 'Pacífica',
    'VALLE DEL CAUCA': 'Pacífica',
    'NARIÑO': 'Pacífica',
    'BOGOTA': 'Andina',
    'CUNDINAMARCA': 'Andina',
    'HUILA': 'Andina',
    'TOLIMA': 'Andina',
    'QUINDIO': 'Andina',
    'RISARALDA': 'Andina',
    'SANTANDER': 'Andina',
    'N. DE SANTANDER': 'Andina',
    'META': 'Orinoquia'}

# Eliminar caracteres no numéricos y convertir a numérico
df['VALOR_PRODUCTO'] = df['VALOR_PRODUCTO'].replace({r'[^\d.]': '', 'INDETERMINADO': None}, regex=True)
df['VALOR_PRODUCTO'] = pd.to_numeric(df['VALOR_PRODUCTO'], errors='coerce')

# Calcular el promedio excluyendo ceros y NaN
promedio = df.loc[df['VALOR_PRODUCTO'] > 0, 'VALOR_PRODUCTO'].mean()

# Imputar los valores de '0' y NaN con el promedio calculado
df['VALOR_PRODUCTO'] = df['VALOR_PRODUCTO'].apply(lambda x: promedio if pd.isna(x) or x == 0 else x)

df['REGION'] = df['UNIDAD_DEPARTAMENTO'].map(REGION_DEPTO)

variables_interes = ['REGION','ATENCION_TEMA', 'PERSONA_RANGO_EDAD', 'PERSONA_GENERO', 'PERSONA_PROFESION', 'TIPO_PRODUCTO', 'VALOR_PRODUCTO', 'DURACION']
data = df[variables_interes].copy()

# Codificar variables categóricas
categorical_columns = ['REGION','ATENCION_TEMA', 'PERSONA_RANGO_EDAD', 'PERSONA_GENERO', 'PERSONA_PROFESION', 'TIPO_PRODUCTO']
data_final = pd.get_dummies(data, columns=categorical_columns).astype(int)

# Identificar las variables independientes
X = data_final.drop('DURACION', axis=1)
# Identificar la variable dependiente
y = data_final['DURACION']

# Dividir la muestra
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=0)

# registre el experimento
experiment = mlflow.set_experiment("random-forest")

# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo.
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # Parámetros del modelo
    n_estimators=46
    max_depth=2
    min_samples_split=3
    min_samples_leaf=4
    random_state=42
    # Cree el modelo con los parámetros definidos y entrénelo
    rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, random_state=42)
    rf.fit(XTrain, yTrain)
    # Realice predicciones de prueba
    predictions = rf.predict(XTest)

    # Registre los parámetros
    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("maxdepth", max_depth)
    mlflow.log_param("min_samples_split", min_samples_split)
    mlflow.log_param("min_samples_leaf", min_samples_leaf)
    mlflow.log_param("random_state", random_state)

    # Registre el modelo
    mlflow.sklearn.log_model(rf, "random-forest-proyecto")

    # Cree y registre la métrica de interés
    mse = mean_squared_error(yTest, predictions)
    mlflow.log_metric("mse", mse)
    print(mse)